In [16]:
from keras_squeezenet import SqueezeNet

from keras.layers import Dropout, Convolution2D, Activation, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

import tensorflow as tf

from collections import Counter

In [17]:
WIDTH, HEIGHT = (664, 485)

DROPOUT=0.2
CLASSES=2
BATCH_SIZE=8
NUM_EPOCHS=20
INIT_LR=0.001

BASE_PATH = 'data/chest_xray/'
TRAIN_PATH = BASE_PATH + 'train'
VAL_PATH = BASE_PATH + 'val'
TEST_PATH = BASE_PATH + 'test'

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=10,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(HEIGHT, WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE)

validation_generator = test_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(HEIGHT, WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(HEIGHT, WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [19]:
counter = Counter(train_generator.classes)                        
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

In [20]:
base_model = SqueezeNet(input_shape=(HEIGHT, WIDTH, 3), weights="imagenet")
x = base_model.output
x = Dropout(DROPOUT, name='drop9')(x)

x = Convolution2D(CLASSES, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
predictions = Activation('softmax', name='loss')(x)

In [21]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = Model(inputs = base_model.input, outputs = predictions)
    model.compile(optimizer=Adam(lr=INIT_LR), loss='categorical_crossentropy', metrics=['accuracy'])
    
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    class_weight=class_weights,
    shuffle=True
)

model.evaluate_generator(generator=test_generator,
                         steps=test_generator.samples // BATCH_SIZE)
    
model.save('model.h5')

Epoch 1/20
  5/652 [..............................] - ETA: 16:38 - loss: 1.3159 - accuracy: 0.6500

KeyboardInterrupt: 